# Adaptive Parking Optimization System (APSO)

This notebook demonstrates an Adaptive Particle Swarm Optimization (APSO) technique to recommend an optimal parking lot near a given destination. It enhances the basic PSO by dynamically adjusting parameters based on the population diversity at each iteration.

**Key Features:**
- Calculates distances using the Haversine formula.
- Scores parking lots based on weighted criteria (distance, cost, and accessibility).
- Dynamically adjusts PSO parameters (`w`, `c1`, `c2`) to balance exploration and exploitation.
- Provides a map visualization (using folium) of recommended parking lots.
- Attempts an extended search radius if no suitable parking is found initially.

**Requirements:**
- A CSV file named `Parking_Lot_Dataset.csv` containing:
  - `id`, `lat`, `lng`, `rate_hourly`

**Instructions:**
1. Update the `destination` variable with your desired coordinates.
2. Run all cells in order to generate recommendations and view results.
3. The map visualization will be saved as `parking_recommendation.html`.


## Imports and Setup

Import necessary libraries and set up the environment.


In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2
import folium
from typing import List, Tuple, Dict
import random

## Class Definition: AdaptiveParkingOptimizer

This class:
- Filters parking lots based on a specified search radius.
- Computes fitness for each lot.
- Implements an Adaptive PSO algorithm where inertia weight (`w`) and acceleration coefficients (`c1`, `c2`) are adjusted based on population diversity.
- Provides methods to run the optimization and visualize the results.


In [2]:
class AdaptiveParkingOptimizer:
    def __init__(self, parking_data: pd.DataFrame, destination: Tuple[float, float],
                 weights: Dict[str, float] = None, search_radius: int = 800):
        """
        Initialize the parking optimizer with Adaptive PSO
        """
        self.parking_data = parking_data
        self.destination = destination
        self.weights = weights or {'distance': 0.5, 'cost': 0.3, 'accessibility': 0.2}
        self.search_radius = search_radius

        # Calculate distances for all parking lots
        self.parking_data['distance'] = self.parking_data.apply(
            lambda row: self.haversine_distance(
                row['lat'], row['lng'],
                self.destination[0], self.destination[1]
            ),
            axis=1
        )

        # Filter parking lots within search radius
        self.filtered_data = self.parking_data[self.parking_data['distance'] <= self.search_radius].copy()

        # PSO parameters
        self.num_particles = min(20, len(self.filtered_data)) if len(self.filtered_data) > 0 else 20
        self.max_iterations = 50

        # APSO parameter ranges
        self.w_max = 0.9
        self.w_min = 0.4
        self.c1_max = 2.5
        self.c1_min = 1.5
        self.c2_max = 2.5
        self.c2_min = 1.5

        # Current parameters
        self.w = self.w_max
        self.c1 = self.c1_max
        self.c2 = self.c2_min

        # Diversity thresholds
        self.diversity_high = 0.7
        self.diversity_low = 0.3

    def haversine_distance(self, lat1: float, lon1: float, lat2: float, lon2: float) -> float:
        """Calculate the distance between two points on Earth"""
        R = 6371000  # Earth's radius in meters
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * atan2(sqrt(a), sqrt(1-a))
        return R * c

    def calculate_fitness(self, parking_lot: pd.Series) -> float:
        """Calculate fitness score for a parking lot"""
        if len(self.filtered_data) == 0:
            return 0

        max_distance = self.filtered_data['distance'].max()
        max_cost = self.filtered_data['rate_hourly'].max()

        distance_score = (max_distance - parking_lot['distance']) / max_distance if max_distance > 0 else 1
        cost_score = (max_cost - parking_lot['rate_hourly']) / max_cost if max_cost > 0 else 1
        accessibility_score = random.uniform(0.7, 1.0)

        fitness = (
            self.weights['distance'] * distance_score +
            self.weights['cost'] * cost_score +
            self.weights['accessibility'] * accessibility_score
        )
        return fitness

    def calculate_diversity(self, positions: List[int]) -> float:
        """Calculate population diversity"""
        if not positions:
            return 0.0
        mean_pos = np.mean(positions)
        diversity = np.std([abs(p - mean_pos) for p in positions]) / len(self.filtered_data) if len(self.filtered_data) > 0 else 0
        return diversity

    def update_parameters(self, iteration: int, diversity: float) -> None:
        """Update PSO parameters based on current state"""
        # Update inertia weight linearly
        self.w = self.w_max - (self.w_max - self.w_min) * iteration / self.max_iterations

        # Update acceleration coefficients based on diversity
        if diversity > self.diversity_high:
            # Exploration phase
            self.c1 = self.c1_max
            self.c2 = self.c2_min
        elif diversity < self.diversity_low:
            # Exploitation phase
            self.c1 = self.c1_min
            self.c2 = self.c2_max
        else:
            # Transition phase
            self.c1 = (self.c1_max + self.c1_min) / 2
            self.c2 = (self.c2_max + self.c2_min) / 2

    def optimize(self) -> Tuple[pd.Series, float, Dict]:
        """Run Adaptive PSO optimization"""
        if len(self.filtered_data) == 0:
            return None, 0, {}

        # Initialize particles
        positions = random.sample(range(len(self.filtered_data)), self.num_particles)
        velocities = [random.uniform(-1, 1) for _ in range(self.num_particles)]
        particle_best_positions = positions.copy()
        particle_best_fitness = [self.calculate_fitness(self.filtered_data.iloc[pos])
                               for pos in positions]

        # Initialize global best
        global_best_idx = np.argmax(particle_best_fitness)
        global_best_position = positions[global_best_idx]
        global_best_fitness = particle_best_fitness[global_best_idx]

        # Optimization history
        history = {
            'diversity': [],
            'best_fitness': [],
            'w': [],
            'c1': [],
            'c2': []
        }

        # Optimization loop
        for iteration in range(self.max_iterations):
            diversity = self.calculate_diversity(positions)
            self.update_parameters(iteration, diversity)

            # Store history
            history['diversity'].append(diversity)
            history['best_fitness'].append(global_best_fitness)
            history['w'].append(self.w)
            history['c1'].append(self.c1)
            history['c2'].append(self.c2)

            for i in range(self.num_particles):
                # Update velocity
                r1, r2 = random.random(), random.random()
                cognitive = self.c1 * r1 * (particle_best_positions[i] - positions[i])
                social = self.c2 * r2 * (global_best_position - positions[i])
                velocities[i] = self.w * velocities[i] + cognitive + social

                # Update position
                new_position = int(positions[i] + velocities[i]) % len(self.filtered_data)
                positions[i] = new_position

                # Update fitness
                fitness = self.calculate_fitness(self.filtered_data.iloc[positions[i]])

                # Update particle best
                if fitness > particle_best_fitness[i]:
                    particle_best_fitness[i] = fitness
                    particle_best_positions[i] = positions[i]

                    # Update global best
                    if fitness > global_best_fitness:
                        global_best_fitness = fitness
                        global_best_position = positions[i]

        return self.filtered_data.iloc[global_best_position], global_best_fitness, history

    def get_recommendations(self) -> pd.DataFrame:
        """Get all parking lots with their fitness scores"""
        if len(self.filtered_data) == 0:
            return pd.DataFrame()

        self.filtered_data['fitness'] = self.filtered_data.apply(self.calculate_fitness, axis=1)
        return self.filtered_data.sort_values('fitness', ascending=False)

    def visualize_results(self, recommendations: pd.DataFrame) -> folium.Map:
        """Create a map visualization"""
        m = folium.Map(location=[self.destination[0], self.destination[1]], zoom_start=15)

        # Add destination marker
        folium.Marker(
            [self.destination[0], self.destination[1]],
            popup='Destination',
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

        # Add search radius circle
        folium.Circle(
            radius=self.search_radius,
            location=[self.destination[0], self.destination[1]],
            color='red',
            fill=True,
            fillOpacity=0.1
        ).add_to(m)

        # Add all parking lots (black markers)
        for _, lot in self.parking_data.iterrows():
            popup_text = f"""
                ID: {lot['id']}<br>
                Rate: ${lot['rate_hourly']}/hr<br>
                Distance: {lot['distance']:.0f}m
            """
            folium.Marker(
                [lot['lat'], lot['lng']],
                popup=popup_text,
                icon=folium.Icon(color='black')
            ).add_to(m)

        # Add recommended parking lots (green for best, blue for others)
        if not recommendations.empty:
            best_lot = recommendations.iloc[0]
            for _, lot in recommendations.iterrows():
                color = 'green' if lot['id'] == best_lot['id'] else 'blue'
                popup_text = f"""
                    ID: {lot['id']}<br>
                    Rate: ${lot['rate_hourly']}/hr<br>
                    Distance: {lot['distance']:.0f}m<br>
                    Fitness Score: {lot['fitness']:.3f}
                """
                folium.Marker(
                    [lot['lat'], lot['lng']],
                    popup=popup_text,
                    icon=folium.Icon(color=color)
                ).add_to(m)

        return m


## Main Function

`parking_recommendation()`:
- Loads the parking data.
- Initializes `AdaptiveParkingOptimizer`.
- Runs the APSO optimization.
- Prints the best and other recommended lots.
- Produces a map of the recommendations.


In [3]:
def parking_recommendation(destination: Tuple[float, float], search_radius: int = 800) -> None:
    """Main function to run parking optimization"""
    parking_data = pd.read_csv('Parking_Lot_Dataset.csv')

    weights = {
        'distance': 0.5,
        'cost': 0.3,
        'accessibility': 0.2
    }

    # Try with initial search radius
    optimizer = AdaptiveParkingOptimizer(parking_data, destination, weights, search_radius)
    best_lot, best_fitness, history = optimizer.optimize()
    recommendations = optimizer.get_recommendations()

    if recommendations.empty:
        print(f"No parking lots are available within {search_radius} meters radius.")
        extended_radius = int(search_radius * 1.5)
        print(f"Trying extended radius of {extended_radius} meters...")
        optimizer = AdaptiveParkingOptimizer(parking_data, destination, weights, extended_radius)
        best_lot, best_fitness, history = optimizer.optimize()
        recommendations = optimizer.get_recommendations()

        if recommendations.empty:
            print(f"No parking lots are available within {extended_radius} meters radius.")
            map_viz = optimizer.visualize_results(recommendations)
            map_viz.save('parking_recommendation.html')
            return

    current_radius = optimizer.search_radius
    print(f"\nBest Parking Lot Found in {current_radius}m:")
    print(f"ID: {best_lot['id']}")
    print(f"Location: ({best_lot['lat']}, {best_lot['lng']})")
    print(f"Hourly Rate: ${best_lot['rate_hourly']}")
    print(f"Distance to destination: {best_lot['distance']:.0f}m")
    print(f"Fitness Score: {best_fitness:.3f}")

    if len(recommendations) > 1:
        print("\nOther Available Parking Lots:")
        for _, lot in recommendations.iloc[1:].iterrows():
            print(f"\nID: {lot['id']}")
            print(f"Distance: {lot['distance']:.0f}m")
            print(f"Rate: ${lot['rate_hourly']}/hr")
            print(f"Fitness Score: {lot['fitness']:.3f}")

    map_viz = optimizer.visualize_results(recommendations)
    map_viz.save('parking_recommendation.html')


## Run the Recommendation

Adjust `destination` and `search_radius` as needed.


In [4]:
downtown = (43.6532, -79.3832)
parking_recommendation(downtown, search_radius=500)


Best Parking Lot Found in 500m:
ID: 16.0
Location: (43.651696, -79.383699)
Hourly Rate: $7.0
Distance to destination: 172m
Fitness Score: 0.559

Other Available Parking Lots:

ID: 15.0
Distance: 408m
Rate: $6.0/hr
Fitness Score: 0.348

ID: 135.0
Distance: 341m
Rate: $8.0/hr
Fitness Score: 0.327

ID: 12.0
Distance: 484m
Rate: $7.0/hr
Fitness Score: 0.215


## Output and Visualization

- The best lot and others (if any) are printed in the cell above.
- A map file `parking_recommendation.html` is saved in the current directory.
